In [48]:
from random import randint
import math

# absolute value of centered modulo of integers
def abs_mod(a, m):
    ret = a % m
    return (ret if ret <= m//2 else m-ret)

# absolute value of centered modulo of real numbers
def abs_mod_float(a, m):
    return abs(a-m*round(a/m))

# modulo of real numbers
def mod_float(a, m):
    return a-m*math.floor(a/m)

# inversion of a (mod n)
def mod_inversion(a, n):
    a, m, k1, k2 = a%n, n, 0, 1
    while a!=0:
        r, d = n%a, n//a
        k1, k2, n, a = k2, k1-d*k2, a, r
    return (k1%m if n==1 else None)

# list_A and list_B must contain disjoint intervals in increasing order
# open intervals are used (endpoints not included)
# inspired by
#   https://geeksforgeeks.org/find-intersection-of-intervals-given-by-two-lists
def intersect_two_interval_lists(list_A, list_B):
    ret = []
    i, j, n, m = 0, 0, len(list_A), len(list_B)
    while i < n and j < m:
        low = max(list_A[i][0], list_B[j][0])
        if list_A[i][1] <= list_B[j][1]:
            top = list_A[i][1]
            i += 1
        else:
            top = list_B[j][1]
            j += 1
        if low < top:
            ret.append((low, top))
    return ret

# list_of_lists: elements are lists of disjoint intervals in increasing order
def intersect_interval_lists(list_of_lists):
    ret = [(-math.inf, math.inf)]
    for alist in list_of_lists:
        ret = intersect_two_interval_lists(ret, alist)
    return ret

# creates a list of all intervals containing real solutions
#   to one HNP inequality
# covers all solutions in the range [0,q]
# some of the returned intervals can possibly exceed this range
def make_interval_list_HNP(q, ti, ui, w):
    period = q/abs(ti)
    b = mod_float(ui/ti + period/(2*w), period)
    a = b - period/w
    k, cur, ret = 0, a, []
    while cur < q:
        ret.append((cur, cur+period/w))
        k += 1
        cur = a + k*period
    return ret

class HNP:
    def __init__(self, q, default=None, li_vs_wi=True,
                 matrix_B_corner=1, alpha=None):
        self.q = q  # should be prime number, not checked
        self.alpha = alpha
        self.d = 0
        self.t = []
        self.u = []
        self.li_vs_wi = li_vs_wi  # using number of "bits" (li) is default
        if li_vs_wi:
            self.default = default if default else 1
        else:
            self.default = default if default else 2
        self.l_or_w_dict = {}
        self.interval_list = None
        self.number_of_intervals = None
        self.solution_list = None
        self.number_of_solutions = None
        self.matrix_B_corner = matrix_B_corner
        self.reduced_HNP = None
    
    def get_w(self, i=-1):
        if self.li_vs_wi:
            return 2**self.l_or_w_dict.get(i, self.default)
        return self.l_or_w_dict.get(i, self.default)
    
    def clear(self):  # back to initial state
        self.__init__(self.q, self.default, self.li_vs_wi,
                      self.matrix_B_corner, self.alpha)
        return self

    def make_reduced_HNP(self):
        reduced_HNP = HNP(self.q, self.default,
                          self.li_vs_wi, self.get_w(self.d-1))
        reduced_HNP.l_or_w_dict = self.l_or_w_dict
        t_1_1 = mod_inversion(self.t[-1], self.q)
        temp = (-self.u[-1]*t_1_1)%self.q
        for i in range(self.d-1):
            reduced_HNP.add_ineq((t_1_1*self.t[i])%self.q,
                                       (self.u[i]+temp*self.t[i])%self.q)
        self.reduced_HNP = reduced_HNP
        return self.reduced_HNP
    
    def solution_from_reduced_HNP(self):
        return ((self.reduced_HNP.alpha+self.u[-1]) \
                 *mod_inversion(self.t[-1],self.q))%self.q
    
    def solutions_from_reduced_HNP(self):
        ret = []
        for sol in self.reduced_HNP.solution_list:
            ret.append(((sol+self.u[-1])* \
                        mod_inversion(self.t[-1],self.q))%self.q)
        return ret
    
    def make_random(self, n=-1):
        self.clear()
        wi = 2**self.default if self.li_vs_wi else self.default
        if n < 0:
            s, n = 1, 0
            while s < self.q:
                s *= wi
                n += 1
        self.alpha = self.alpha if self.alpha else randint(1,self.q-1)
        self.t = [randint(1,self.q-1) for _ in range(n)]
        self.u = [(self.alpha*self.t[i]+randint(math.floor(-self.q/(2*wi)+1),
                  math.ceil(self.q/(2*wi)-1)))%self.q for i in range(n)]
        self.d = n
        return self
    
    def check_solution(self, int_number):
        return all([abs_mod(int_number*self.t[i]-self.u[i], \
                    self.q)*2*self.get_w(i) < self.q for i in range(self.d)])
    
    def check_solution_float(self, float_number):
        return all([abs_mod_float(float_number*self.t[i]-self.u[i], \
                    self.q)*2*self.get_w(i) < self.q for i in range(self.d)])
    
    def make_matrix_B(self):
        return matrix([[(2*self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+1)] \
                       for i in range(self.d)] + \
                [[2*self.get_w(j)*self.t[j] for j in range(self.d)] + \
                 [self.matrix_B_corner]])
    
    def make_matrix_C(self):  # C is called B' in the thesis ???
        return matrix([[(2*self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+2)] \
                       for i in range(self.d)] + \
            [[2*self.get_w(j)*self.t[j] for j in range(self.d)]+ \
             [self.matrix_B_corner, 0]] + \
            [[2*self.get_w(j)*self.u[j] for j in range(self.d)]+[0, self.q]])
    
    def add_ineq(self, ti, ui, li_or_wi=None):
        if li_or_wi and li_or_wi!=self.default:
            self.l_or_w_dict[self.d] = li_or_wi
        self.t.append(ti)
        self.u.append(ui)
        self.d += 1
        return self
    
    def pop_ineq(self):
        self.d -= 1
        return (self.t.pop(), self.u.pop(),
                self.l_or_w_dict.pop(self.d, self.default))
    
    def make_intervals(self, find_integers=True):
        self.interval_list = intersect_interval_lists( \
            [make_interval_list_HNP(self.q, self.t[i], self.u[i], self.get_w(i)) \
             for i in range(self.d)]+[[(0,self.q)]])
        self.number_of_intervals = len(self.interval_list)
        if self.interval_list[0][0]==0 and self.interval_list[-1][1]==self.q \
                and self.check_solution(0):
            self.number_of_intervals -= 1
        if find_integers:
            self.integer_solution_from_intervals()
        return self
    
    def bruteforce_integer_solution(self):
        ret = []
        for i in range(self.q):
            if self.check_solution(i):
                ret.append(i)
        self.solution_list = ret
        self.number_of_solutions = len(ret)
        return self
    
    def integer_solution_from_intervals(self):
        assert self.interval_list
        ret = []
        if self.check_solution(0):
            ret.append(0)
        for r in self.interval_list:
            ret.extend(range(math.floor(r[0]+1), math.ceil(r[1])))
        if self.solution_list is None:
            self.number_of_solutions = len(ret)
            self.solution_list = ret
        else:
            assert self.number_of_solutions == len(ret)
            assert self.solution_list == ret
        return self
    
    # yields solutions to two HNP inequalities (at indices i, j)
    # the yielded solutions are not ordered
    def two_ineq_solution_generator(self, i=0, j=1):
        c1 = ceil(self.q/(2*self.get_w(i))-1)
        c2 = ceil(self.q/(2*self.get_w(j))-1)
        t1_1 = mod_inversion(self.t[i], self.q)
        u1_bar, u2_bar = self.u[i]-c1, self.u[j]-c2
        tk = (t1_1*self.t[j])%self.q
        vk = (-u1_bar*t1_1*self.t[j]+u2_bar)%self.q
        for x1 in self.decentered_ineq_solution_generator(self.q,tk,vk,2*c2):
            if x1 <= 2*c1:
                yield ((x1+u1_bar)*t1_1)%self.q
            else:
                break
    
    # yielded values are increasing non-negative integers
    # assuming li is at least 1 bit ???
    def ineq_solution_generator(self, q, ti, ui, wi):
        r = 2*ceil(q/(2*wi)-1)
        vi = (ui-r//2)%q
        yield from self.decentered_ineq_solution_generator(q, ti, vi, r)
    
    # yields solutions to the given decentered HNP inequality
    # solutions are yielded in increasing order
    # assuming 0 <= r < n/2
    # intentionally unoptimized (aim for shorter code)
    def decentered_ineq_solution_generator(self, n, ti, vi, r):
        if ti > n//2:
            ti, vi = n-ti, (n-vi-r)%n
        b = 0
        while True:
            while (b*ti-vi)%n <= r:
                yield b
                b += 1
            b += ((n-(b*ti-vi)%n)-1)//ti+1
            if (b*ti-vi)%n > r:
                break
        s = ((n-1)//ti+1)
        t_new = s*ti-n
        for x in self.decentered_ineq_solution_generator(ti, t_new,
                                                         (vi-(b-1)*ti)%n, r):
            yield b-1 + x*s - (x*t_new)//ti



In [49]:
h = HNP(1999,3)
h.make_random(3)
h.make_reduced_HNP()
h.bruteforce_integer_solution()
h.make_intervals()
h.integer_solution_from_intervals()
h.make_reduced_HNP()
h.reduced_HNP.bruteforce_integer_solution()
h.reduced_HNP.make_intervals()
h.reduced_HNP.integer_solution_from_intervals()

In [92]:
print(h.solutions_from_reduced_HNP())

[512, 644, 570, 496, 422, 480, 406, 332, 390, 316, 226, 136, 46, 1274, 1184, 1094, 1020, 1078, 1004, 930, 988, 914, 840, 766, 898, 824, 750, 676, 734, 660, 586]


In [51]:
h.solution_list

[422, 496, 512, 570, 586, 644, 660, 734]

In [52]:
h.make_matrix_C()

[31984     0     0     0     0]
[    0 31984     0     0     0]
[    0     0 31984     0     0]
[28080 22032  3888     1     0]
[14256 20320  7632     0  1999]

In [53]:
h.make_intervals()

In [55]:
h.integer_solution_from_intervals()

In [56]:
h.solution_list

[422, 496, 512, 570, 586, 644, 660, 734]

In [57]:
h.number_of_solutions

8

In [58]:
h.number_of_intervals

81

In [91]:
print(h.reduced_HNP.solution_list)

[1, 93, 102, 111, 120, 221, 230, 239, 340, 349, 468, 587, 706, 1259, 1378, 1497, 1506, 1607, 1616, 1625, 1726, 1735, 1744, 1753, 1845, 1854, 1863, 1872, 1973, 1982, 1991]


In [81]:
h.reduced_HNP.alpha = 120

In [82]:
h.solution_from_reduced_HNP()

422

In [62]:
h.reduced_HNP.make_matrix_B()

[31984     0     0]
[    0 31984     0]
[24992 10752     8]

In [63]:
h.reduced_HNP.make_matrix_C()

[31984     0     0     0]
[    0 31984     0     0]
[24992 10752     8     0]
[23104  9056     0  1999]

In [72]:
h.check_solution(570)

True

In [83]:
h.check_solution_float(422)

True

In [84]:
while True:
    n = HNP(1999,3)
    n.make_random(3)
    if n.check_solution(0):
        break


In [85]:
n.make_intervals().integer_solution_from_intervals()

In [86]:
print(n.solution_list)

[0, 5, 10, 15, 20, 1002, 1007, 1012, 1017]


In [87]:
n.number_of_intervals

52

In [88]:
len(n.interval_list)

53

In [49]:
p = HNP(11,0.4)

In [50]:
p.make_random(2)

In [51]:
for val in p.two_ineq_solution_generator():
    print(val, end=", ")

5, 9, 2, 4, 6, 8, 10, 

In [52]:
p.make_intervals().interval_list

[(0.228972430262051, 0.861370092983932),
 (1.60397243026205, 2.64602756973795),
 (3.13862990701607, 4.02102756973795),
 (4.35397243026205, 4.52803675965060),
 (4.97196324034940, 5.39602756973795),
 (5.72897243026205, 6.36137009298393),
 (7.10397243026205, 8.14602756973795),
 (8.63862990701607, 9.52102756973795),
 (9.85397243026205, 10.0280367596506),
 (10.4719632403494, 10.8960275697379)]

In [53]:
p.solution_list

[2, 4, 5, 6, 8, 9, 10]

In [54]:
p.bruteforce_integer_solution()

In [55]:
print(p.solution_list)
print(p.get_w())

[2, 4, 5, 6, 8, 9, 10]
1.31950791077289


In [57]:
p.number_of_intervals

10

In [58]:
for y in p.ineq_solution_generator(17, 5, 16, 7):
    if y>25:
        break
    print(y, end=", ")

0, 3, 10, 17, 20, 

In [59]:
for y in p.ineq_solution_generator(17, 5, 15, 7):
    if y>25:
        break
    print(y, end=", ")

3, 10, 13, 20, 

In [60]:
r = HNP(17, 7, False)
r.add_ineq(5,15,7)

In [61]:
r.t

[5]

In [62]:
r.make_intervals()

In [63]:
r.solution_list

[3, 10, 13]

In [64]:
r.bruteforce_integer_solution()

In [65]:
r.solution_list

[3, 10, 13]

In [66]:
for i in range(17):
    print(i, r.check_solution(i))

0 False
1 False
2 False
3 True
4 False
5 False
6 False
7 False
8 False
9 False
10 True
11 False
12 False
13 True
14 False
15 False
16 False
